# Collect temperature data in a specified area of the globe

In [18]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import itertools
import requests
import time
from datetime import datetime as dt, date, timedelta

# Import API key
from api_keys import weather_api_key

### Generate List of Lat/Longs

In [28]:
#Define our lat/lon range and increments between the range
# min_lat = 32
# max_lat = 42
# min_lon = -125
# max_lon = -114
# increment = .3

min_lat = 33
max_lat = 43
min_lon = 134
max_lon = 145
increment = 0.3

#Build our lat/lon list within range
lat_lons = []

lats = [round(num, 2) for num in np.arange(min_lat, max_lat, increment).tolist()]
lons = [round(num, 2) for num in np.arange(min_lon, max_lon, increment).tolist()]
lat_lons = list(itertools.product(lats, lons))
len(lat_lons)

1258

### Define date of earthquake and calculate the date before the earthquake

In [29]:
import datetime
day_of_eq = "2011-03-11"
day_of_eq_dtformat = dt.strptime(day_of_eq,'%Y-%m-%d')
day_before_eq_dtformat = day_of_eq_dtformat - datetime.timedelta(days=1)
day_before_eq=dt.strftime(day_before_eq_dtformat,'%Y-%m-%d')
day_before_eq

'2011-03-10'

### Pull temperature data for each lat/long using a series of successive API calls.

In [30]:
appid = weather_api_key

In [31]:
base_url = "http://api.weatherstack.com/historical?"

counter = 1
city_data = []

for lat_lon in lat_lons:
    params = {
      "query" : f'{lat_lon[0]},{lat_lon[1]}',
      "historical_date_start" : day_before_eq,
      "historical_date_end" : day_of_eq,  
      "hourly" : 0,
      "units" : "f",
      "access_key" : appid
       }

    weather = requests.get(base_url, params=params).json()
    
    lat = lat_lon[0]
    lon = lat_lon[1]
    temp1 = weather["historical"][day_of_eq]["avgtemp"]
    temp0 = weather["historical"][day_before_eq]["avgtemp"]
    date_of_eq = weather["historical"][day_of_eq]["date"]
    date_before_eq = weather["historical"][day_before_eq]["date"]
    city = weather["location"]["name"]
    country = weather["location"]["country"]

    city_data.append({"City" : city
                    ,"Country" : country
                    ,"Latitude" : lat
                    ,"Longitude" : lon
                    ,f'Avg Temperature {date_before_eq}': temp0
                    ,f'Avg Temperature {date_of_eq}': temp1
                    ,"Date": date_of_eq
                    })

    print(f"Processing Record {counter} - {city}")
    counter = counter + 1


Processing Record 1 - Tsuro
Processing Record 2 - Tsuro
Processing Record 3 - Tsuro
Processing Record 4 - Mitsu
Processing Record 5 - Hiki
Processing Record 6 - Uwano
Processing Record 7 - Uwano
Processing Record 8 - Sue
Processing Record 9 - Kashino
Processing Record 10 - Kashino
Processing Record 11 - Taiji
Processing Record 12 - Taiji
Processing Record 13 - Nakiri
Processing Record 14 - Nakiri
Processing Record 15 - Toriuchi
Processing Record 16 - Toriuchi
Processing Record 17 - Toriuchi
Processing Record 18 - Toriuchi
Processing Record 19 - Toriuchi
Processing Record 20 - Toriuchi
Processing Record 21 - Kamijinoo
Processing Record 22 - Kamijinoo
Processing Record 23 - Kamijinoo
Processing Record 24 - Kamijinoo
Processing Record 25 - Kamijinoo
Processing Record 26 - Kamijinoo
Processing Record 27 - Kamijinoo
Processing Record 28 - None
Processing Record 29 - None
Processing Record 30 - None
Processing Record 31 - None
Processing Record 32 - None
Processing Record 33 - None
Processin

In [32]:
print(weather)

{'request': {'type': 'LatLon', 'query': 'Lat 42.90 and Lon 144.80', 'language': 'en', 'unit': 'f'}, 'location': {'name': 'Besshakudomari', 'country': 'Japan', 'region': 'HokkaidÅ\x8d', 'lat': '42.940', 'lon': '144.777', 'timezone_id': 'Asia/Tokyo', 'localtime': '2020-08-17 16:27', 'localtime_epoch': 1597681620, 'utc_offset': '9.0'}, 'current': {'observation_time': '07:27 AM', 'temperature': 72, 'weather_code': 176, 'weather_icons': ['https://assets.weatherstack.com/images/wsymbols01_png_64/wsymbol_0009_light_rain_showers.png'], 'weather_descriptions': ['Patchy rain possible'], 'wind_speed': 2, 'wind_degree': 179, 'wind_dir': 'S', 'pressure': 1007, 'precip': 0, 'humidity': 82, 'cloudcover': 34, 'feelslike': 72, 'uv_index': 5, 'visibility': 6, 'is_day': 'yes'}, 'historical': {'2011-03-10': {'date': '2011-03-10', 'date_epoch': 1299715200, 'astro': {'sunrise': '05:43 AM', 'sunset': '05:20 PM', 'moonrise': '07:39 AM', 'moonset': '10:48 PM', 'moon_phase': 'Waxing Crescent', 'moon_illuminatio

### Convert the temperature data into a Dataframe

In [33]:
df = pd.DataFrame(city_data)

In [34]:
# Insert a column in the dataframe named "Temp_Difference" which contains the difference between 
# the temperature the day of the earthquakeand the day before the earthquake.

df['Temp_Difference']=abs(df[f'Avg Temperature {date_of_eq}'] - df[f'Avg Temperature {date_before_eq}'])
df

,City,Country,Latitude,Longitude,Avg Temperature 2011-03-10,Avg Temperature 2011-03-11,Date,Temp_Difference
0,Tsuro,Japan,33.0,134.0,43,48,2011-03-11,5
1,Tsuro,Japan,33.0,134.3,43,48,2011-03-11,5
2,Tsuro,Japan,33.0,134.6,43,48,2011-03-11,5
3,Mitsu,Japan,33.0,134.9,43,48,2011-03-11,5
4,Hiki,Japan,33.0,135.2,43,48,2011-03-11,5
...,...,...,...,...,...,...,...,...
1253,Ushishubetsu,Japan,42.9,143.6,21,21,2011-03-11,0
1254,Kombukariishi,Japan,42.9,143.9,21,21,2011-03-11,0
1255,Shakubetsu,Japan,42.9,144.2,23,25,2011-03-11,2
1256,Katsurakoi,Japan,42.9,144.5,23,25,2011-03-11,2


### Convert the dataframe into a json file

In [35]:
df.to_json(r'C:\Users\rona_\Desktop\Rona USC Class\Projects\Positive Plotly Pegasus\Earthquake\data\py\Jap_temps_2011Mar11.json',orient="records")